<a href="https://colab.research.google.com/github/MaxTheMooshroom/DS-Unit-2-Applied-Modeling/blob/master/DS_Sprint_Challenge_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_

# Applied Modeling Sprint Challenge: Predict Chicago food inspections 🍔

For this Sprint Challenge, you'll use a dataset with information from inspections of restaurants and other food establishments in Chicago from January 2010 to March 2019. 

[See this PDF](https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF) for descriptions of the data elements included in this dataset.

According to [Chicago Department of Public Health — Food Protection Services](https://www.chicago.gov/city/en/depts/cdph/provdrs/healthy_restaurants/svcs/food-protection-services.html), "Chicago is home to 16,000 food establishments like restaurants, grocery stores, bakeries, wholesalers, lunchrooms, mobile food vendors and more. Our business is food safety and sanitation with one goal, to prevent the spread of food-borne disease. We do this by inspecting food businesses, responding to complaints and food recalls." 

#### Your challenge: Predict whether inspections failed

The target is the `Fail` column.

- When the food establishment failed the inspection, the target is `1`.
- When the establishment passed, the target is `0`.

In [0]:
# Didn't finish. I'd rather not even turn this in, but I am because I must. 

#### Run this cell to install packages in Colab:

In [0]:
%%capture

# Install packages in Colab
!pip install category_encoders==2.*
!pip install eli5
!pip install pandas-profiling==2.*
!pip install pdpbox
!pip install shap

#### Run this cell to load the data:

In [0]:
import pandas as pd

train_url = 'https://drive.google.com/uc?export=download&id=13_tP9JpLcZHSPVpWcua4t2rY44K_s4H5'
test_url  = 'https://drive.google.com/uc?export=download&id=1GkDHjsiGrzOXoF_xcYjdzBTSjOIi3g5a'

_train = pd.read_csv(train_url)
test  = pd.read_csv(test_url)

assert _train.shape == (51916, 17)
assert test.shape  == (17306, 17)

### Part 1: Preprocessing

You may choose which features you want to use, and whether/how you will preprocess them. If you use categorical features, you may use any tools and techniques for encoding.

_To earn a score of 3 for this part, find and explain leakage. The dataset has a feature that will give you an ROC AUC score > 0.90 if you process and use the feature. Find the leakage and explain why the feature shouldn't be used in a real-world model to predict the results of future inspections._

### Part 2: Modeling

**Fit a model** with the train set. (You may use scikit-learn, xgboost, or any other library.) Use cross-validation or do a three-way split (train/validate/test) and **estimate your ROC AUC** validation score.

Use your model to **predict probabilities** for the test set. **Get an ROC AUC test score >= 0.60.**

_To earn a score of 3 for this part, get an ROC AUC test score >= 0.70 (without using the feature with leakage)._


### Part 3: Visualization

Make visualizations for model interpretation. (You may use any libraries.) Choose two of these types:

- Confusion Matrix
- Permutation Importances
- Partial Dependence Plot, 1 feature isolation
- Partial Dependence Plot, 2 features interaction
- Shapley Values

_To earn a score of 3 for this part, make four of these visualization types._

## Part 1: Preprocessing

> You may choose which features you want to use, and whether/how you will preprocess them. If you use categorical features, you may use any tools and techniques for encoding.

In [0]:
# Didn't finish. I'd rather not even turn this in, but I am because I must. 

In [20]:
_train.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Violations,Latitude,Longitude,Location,Fail
0,2088270,"TOM YUM RICE & NOODLE, INC.",TOM YUM CAFE,2354911.0,Restaurant,Risk 1 (High),608 W BARRY,CHICAGO,IL,60657.0,2017-09-15T00:00:00,Canvass,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.938007,-87.644755,"{'longitude': '-87.6447545707008', 'latitude':...",1
1,555268,FILLING STATION & CONVENIENCE STORE,FILLING STATION & CONVENIENCE STORE,1044901.0,Grocery Store,Risk 3 (Low),6646-6658 S WESTERN AVE,CHICAGO,IL,60636.0,2011-10-20T00:00:00,Complaint Re-Inspection,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.772402,-87.683603,"{'longitude': '-87.68360273081268', 'latitude'...",0
2,1751394,A P DELI,A P DELI,47405.0,Restaurant,Risk 1 (High),2025 E 75TH ST,CHICAGO,IL,60649.0,2016-04-05T00:00:00,Canvass Re-Inspection,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.758779,-87.575054,"{'longitude': '-87.57505446746121', 'latitude'...",0
3,1763905,FRANK'S CHICAGO SHRIMP HOUSE,FRANK'S CHICAGO SHRIMP HOUSE,6414.0,Restaurant,Risk 2 (Medium),4459 S ARCHER AVE,CHICAGO,IL,60632.0,2016-04-29T00:00:00,Canvass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.812181,-87.707125,"{'longitude': '-87.70712481334274', 'latitude'...",0
4,453326,MORRILL,MORRILL,24571.0,School,Risk 1 (High),6011 S Rockwell (2600W) AVE,CHICAGO,IL,60629.0,2011-01-10T00:00:00,Canvass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",NaN,NaN,NaN,0


In [21]:
print(_train.columns.to_list())

['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type', 'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date', 'Inspection Type', 'Violations', 'Latitude', 'Longitude', 'Location', 'Fail']


In [0]:
target = 'Fail'
features = ['DBA Name', 'Facility Type', 'Risk', 'Inspection Date', 
            'Inspection Type', 'Violations', 'Zip']

In [22]:
_train['Location'].value_counts()

{'longitude': '-87.91442843927047', 'latitude': '42.008536400868735', 'needs_recoding': False}    741
{'longitude': '-87.74164564419637', 'latitude': '41.789329323265385', 'needs_recoding': False}    159
{'longitude': '-87.68184949426895', 'latitude': '41.88743405025222', 'needs_recoding': False}     136
{'longitude': '-87.6397586848809', 'latitude': '41.88199433820508', 'needs_recoding': False}      121
{'longitude': '-87.63101044588599', 'latitude': '41.88458626715456', 'needs_recoding': False}      99
                                                                                                 ... 
{'longitude': '-87.7251323212337', 'latitude': '41.918903243260104', 'needs_recoding': False}       1
{'longitude': '-87.66819023693519', 'latitude': '41.97833437097476', 'needs_recoding': False}       1
{'longitude': '-87.63963529270472', 'latitude': '41.92448461231529', 'needs_recoding': False}       1
{'longitude': '-87.68466009621312', 'latitude': '41.94325046324114', 'needs_recodi

## Part 2: Modeling

> **Fit a model** with the train set. (You may use scikit-learn, xgboost, or any other library.) Use cross-validation or do a three-way split (train/validate/test) and **estimate your ROC AUC** validation score.
>
> Use your model to **predict probabilities** for the test set. **Get an ROC AUC test score >= 0.60.**

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score

In [25]:
train, validate = train_test_split(_train)
train.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Violations,Latitude,Longitude,Location,Fail
23167,1751986,CAFE EL TAPATIO,CAFE EL TAPATIO,2432567.0,Restaurant,Risk 1 (High),3400-3402 N ASHLAND AVE,CHICAGO,IL,60657.0,2016-04-14T00:00:00,License,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.943432,-87.668913,"{'longitude': '-87.66891275635102', 'latitude'...",0
15546,1938202,ZACHI INC,ZACHI,2172713.0,Restaurant,Risk 1 (High),1406 W BELMONT AVE,CHICAGO,IL,60657.0,2016-07-07T00:00:00,Canvass,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.939869,-87.664171,"{'longitude': '-87.66417066625274', 'latitude'...",0
33159,1589001,THE WINCHESTER,THE WINCHESTER,2298687.0,Restaurant,Risk 1 (High),1001 N WINCHESTER AVE,CHICAGO,IL,60622.0,2015-11-30T00:00:00,Canvass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.899664,-87.675787,"{'longitude': '-87.67578654766524', 'latitude'...",0
9501,1560122,Billy Goat,Billy Goat,1579789.0,Restaurant,Risk 2 (Medium),222 W MERCHANDISE MART PLZ,CHICAGO,IL,60654.0,2015-07-13T00:00:00,Canvass,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.888074,-87.634955,"{'longitude': '-87.63495520292739', 'latitude'...",0
1347,164390,SUBWAY,SUBWAY,2026339.0,Restaurant,Risk 1 (High),1608 W 59TH ST,CHICAGO,IL,60636.0,2010-03-29T00:00:00,License,NaN,41.786850,-87.664801,"{'longitude': '-87.66480141216883', 'latitude'...",1


In [0]:
X_train = train[features]
y_train = train[target]

X_validate = validate[features]
y_validate = validate[target]

X_test = test[features]
y_test = test[target]

In [0]:
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import sklearn

In [0]:
pipeline = make_pipeline(
  ce.OrdinalEncoder(),
  SimpleImputer(missing_values=np.NaN, strategy='most_frequent'),
  RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=2)
)

In [101]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['DBA Name', 'Facility Type', 'Risk',
                                      'Inspection Date', 'Inspection Type',
                                      'Violations'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'DBA Name',
                                          'data_type': dtype('O'),
                                          'mapping': CAFE EL TAPATIO                     1
ZACHI INC                           2
THE WINCHESTER                      3
Billy Goat                          4
SUBWAY                              5
                                ...  
DEL SUL BAR & GRILL LIMIT...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=7,
                     

In [102]:
pipeline.score(X_validate, y_validate)

0.7410432236690038

In [0]:
probas = pipeline.predict_proba(X_validate)
y_pred = pipeline.predict(X_validate)

In [134]:
scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='roc_auc')
scores.mean()

0.692866489482258

In [142]:
confusion_matrix(y_validate, y_pred)

array([[9617,    0],
       [3361,    1]])

In [143]:
probas

array([[0.80653153, 0.19346847],
       [0.77375196, 0.22624804],
       [0.79565323, 0.20434677],
       ...,
       [0.78163674, 0.21836326],
       [0.85860487, 0.14139513],
       [0.74357125, 0.25642875]])

In [145]:
roc_auc_score(confusion_matrix(y_validate, y_pred), probas[:,:-1])

ValueError: ignored

## Part 3: Visualization

> Make visualizations for model interpretation. (You may use any libraries.) Choose two of these types:
>
> - Permutation Importances
> - Partial Dependence Plot, 1 feature isolation
> - Partial Dependence Plot, 2 features interaction
> - Shapley Values

In [33]:
import eli5
from eli5.sklearn import PermutationImportance

from xgboost.sklearn import XGBClassifier

Using TensorFlow backend.


In [0]:
encoder = ce.OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)

imputer = SimpleImputer()
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_encoded), index=X_train.index, columns=X_train.columns)

In [117]:
X_train_imputed.describe()

,DBA Name,Facility Type,Risk,Inspection Date,Inspection Type,Violations,Zip
count,38937.000000,38937.000000,38937.000000,38937.000000,38937.000000,38937.000000,38937.000000
mean,5808.839227,5.311683,1.352082,1016.063744,3.066723,12882.249274,60628.907136
std,4266.058683,20.842571,0.621517,607.850785,2.537659,10286.789879,22.100263
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,60007.000000
25%,2076.000000,1.000000,1.000000,491.000000,2.000000,2498.000000,60614.000000
50%,5125.000000,1.000000,1.000000,995.000000,2.000000,12207.000000,60625.000000
75%,9100.000000,2.000000,2.000000,1522.000000,4.000000,21926.000000,60643.000000
max,15266.000000,300.000000,4.000000,2245.000000,46.000000,31654.000000,60827.000000


In [0]:
perm = PermutationImportance(pipeline, random_state=42)
perm.fit(X_train_imputed, y_train);

In [118]:
eli5.show_weights(perm, feature_names=X_train_imputed.columns.str.replace(' ', '_').to_list())

# I have no idea what or where the error is?? My suspicion is that the results of a math
# expression are so low they get rounded to 0, but I'm unsure if that's true or how to fix it. 

/usr/local/lib/python3.6/dist-packages/eli5/formatters/html.py:235: RuntimeWarning: invalid value encountered in double_scalars
  rel_weight = (abs(weight) / weight_range) ** 0.7


Weight,Feature
0 ± 0.0000,Zip
0 ± 0.0000,Violations
0 ± 0.0000,Inspection_Type
0 ± 0.0000,Inspection_Date
0 ± 0.0000,Risk
0 ± 0.0000,Facility_Type
0 ± 0.0000,DBA_Name


In [119]:
#shows if a column contains None or NaN, trying to find the error seen above
for col in X_train_imputed.columns: 
  t = X_train_imputed[col].value_counts(dropna=False).index
  print(col, None in t or np.NaN in t)

DBA Name False
Facility Type False
Risk False
Inspection Date False
Inspection Type False
Violations False
Zip False


In [0]:
boost = XGBClassifier(max_depth=50, n_jobs=-1, random_state=42)

In [122]:
boost.fit(X_train_imputed, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [129]:
boost.score(pd.DataFrame(imputer.transform(encoder.transform(X_validate)), index=X_validate.index, columns=X_validate.columns), y_validate)

0.7328761846059019

In [0]:
# Didn't finish. I'd rather not even turn this in, but I am because I must. 